In [31]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating project at `~/Documents/eth_courses/notebooks/probability`


In [32]:
# Pkg.add("GLMakie")
using GLMakie

GLMakie.activate!(inline=false)

In [33]:
using Random, Distributions
Random.seed!(123)

TaskLocalRNG()

## Process model

In [34]:
function process_model(x_k, r)
    # noise model
    function process_noise_model(r)
        # prob of catgeories 
        p = [r, 1-r]

        categorical_distrib = Categorical(p)

        # draw samples
        sampl = rand(categorical_distrib)
        
        # assign category
        if sampl==1
            v = 1
        else 
            v = -1
        end
        
        return v
    end 
    
    # motion model
    f = x_k + process_noise_model(r)
    
    return f
end

process_model (generic function with 1 method)

In [35]:
function get_position(x,N)
    return x/N
end

get_position (generic function with 1 method)

In [36]:
x₀ = 0
r = 0.4

process_model(x₀, r)

-1

## Measurement model

In [37]:
function measurement_model(x_k,N,L,e)
    θ = 2*π*(x_k/ N)
    
    # true measurement model
    h_true = sqrt((L -cos(θ)^2) + sin(θ)^2)
    
    # measurement noise
    noise_model = Uniform(-e,e)
    w = rand(noise_model)
    
    return h = h_true + w
end

measurement_model (generic function with 1 method)

In [38]:
N = 10
L = 5
e = 0.1

measurement_model(x₀,N,L,e)

2.0173613514906696

# Simulation

### Initialization

In [39]:
# fixed parameters
N = 100
x₀ = N/4

# equal probability for all states in the beginning
p_0 = (1/N) * ones(N);

## Case 1: 

In [40]:
# process params
L = 2
r = 0.5
e = 0.5

# k: no. of timesteps
function simulate(k, x₀)
    
    println("Initial state: ", x₀)
    
    # set initial state
    x = x₀
        
    # preallocate 
    x_vec = zeros(k)
    t_vec = zeros(k)
    
    for i in 1:k
        x_vec[i] = process_model(x, r)
        t_vec[i] = i
    end
    
    return x_vec, t_vec
end


simulate (generic function with 1 method)

In [41]:
x_vec, t_vec = simulate(100, x₀);

Initial state: 25.0


In [42]:
function plot_sim(t_vec, x_vec)
    fig = Figure()
    
    axis_limits = (xlow, xhigh, ylow, high, zlow, zhigh) = (0, N, 0, 1, -1, 1)
    

    ax = Axis3(fig[1, 1], aspect = (1, 1, 1), title = "Visualization", limits = axis_limits, xlabelsize=20,
                xticklabelsize=20, yticklabelsize=20, zticklabelsize=20)

    lines!(ax, t_vec, get_position(x_vec,N), linewidth = 5)
    display(fig)
end

plot_sim (generic function with 1 method)

In [43]:
plot_sim(t_vec, x_vec)

GLMakie.Screen(...)

# Estimator 

In [44]:
# to store prior probabilities 
prior_vec = zeros(N);

# to store posterion probabilities 
posterior_vec = zeros(N);

In [30]:
temp = zeros(N)
a_k= (1/N) * ones(N);

25.0